<a href="https://colab.research.google.com/github/Milyeong/ML_ipynb/blob/main/tflite_model_maker_%EB%8C%93%EA%B8%80_%EC%8A%A4%ED%8C%B8_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8_%EB%B9%8C%EB%93%9C_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 댓글 스팸 머신러닝 모델 빌드
가이드 라인: https://codelabs.developers.google.com/tflite-nlp-model?hl=ko&authuser=1#0


이해를 돕기 위해 가이드 라인 내 설명을 따라섰으며, 이모티콘을 붙인 부분은 따로 덧붙인 부분이다.

## 1.시작하기 전에

실습 내용


*   Google Colab을 사용하여 TensorFlow Lite Model Maker를 설치하는 방법
*   클라우드 서버에서 기기로 데이터를 다운로드하는 방법
* 데이터 로더 사용 방법
* 모델 빌드 방법



## 2.TensorFlow Lite Model Maker 설치

In [ ]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 577 kB 2.1 MB/s 
     |████████████████████████████████| 60.2 MB 1.2 MB/s 
     |████████████████████████████████| 238 kB 58.4 MB/s 
     |████████████████████████████████| 128 kB 59.8 MB/s 
     |████████████████████████████████| 1.1 MB 64.0 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 10.9 MB 35.1 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 840 kB 52.5 MB/s 
     |████████████████████████████████| 3.4 MB 45.3 MB/s 
     |████████████████████████████████| 1.3 MB 51.8 MB/s 
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 99 kB 9.7 MB/s 
     |████████████████████████████████| 352 kB 52.3 MB/s 
     |████████████████████████████████| 40 kB 6.2 MB/s 
     |████████████████████████████████| 1.1 MB 29.2 MB/s 
     |████████████████████████████████| 213 kB 69.1 MB/s 


## 3.코드 가져오기
Modle Maker를 사용하기 위한 요구사항인 TensorFlow 2.x를 실행 중인지 확인합니다.

In [ ]:
import numpy as np
import os
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker.text_classifier import DataLoader
from tflite_model_maker import text_classifier

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

## 4.데이터 다운로드
Cloud 서버에서 기기로 데이터를 다운로드하고 data_file가 로컬파일을 가리키도록 설정합니다.

In [ ]:
data_file = tf.keras.utils.get_file(fname='comment-spam.csv',
                                    origin='https://storage.googleapis.com/laurencemoroney-blog.appspot.com/lmblog_comments.csv',
                                    extract=False)

245760/235543 [===============================] - 0s 0us/step


Model Maker는 이와 같은 간단한 CSV 파일로 모델을 학습시킬 수 있습니다. 텍스트를 저장할 열과 라벨을 보존하기만 하면 됩니다. Codelab의 뒷부분에서 그 방법을 알아봅시다.

## 5.사전 학습된 임베딩
일반적으로 Model Maker를 사용할 때는 모델을 처음부터 빌드하지 않습니다. 필요에 따라 맞춤설정한 기존모델을 사용합니다.

이와 같은 언어 모델의 선행 학습된 엠베딩 사용이 포함됩니다. 임베딩의 기본은 단어들이 숫자로 이루어져 있으며, 전체 자료에서 각 단어가 숫자로 변형된다는 것입니다. 임베딩은 단어의 '방향'을 설정하여 단어의 감정을  확인하는 데 사용되는 벡터입니다. 예를 들어 댓글 스팸 메시지에 자주 사용되는 단어는 벡터가 비슷한 방향을 가리키고 벡터가 반대 방향을 가리키게 됩니다.

선행 학습된 임베딩을 사용하면 이미 대량의 텍스트에서 감정을 학습한 코퍼스 또는 컬렉션으로 시작할 수 있습니다. 이렇게 하면 0부터 시작하는 것보다 훨씬 빠른 해결 방법을 얻을 수 있습니다.

Model Maker는 사용할 수 있는 몇 가지 선행 학습된 임베딩을 제공하지만 가장 간단하고 빠르게 시작하는 방법은 average_word_vec입니다.

코드는 다음과 같습니다.

In [ ]:
spec = model_spec.get('average_word_vec')
spec.num_words = 2000
spec.seq_len = 20
spec.wordvec_dim = 7

### num_words 매개변수
모델에서 사용할 단어 수를 지정합니다.(You will also specify the number of words you want your model to use.)

'많을수록 좋습니다'라고 생각할 수 있지만 일반적으로 각 단어가 사용되는 빈도에 따라 알맞은 숫자가 있습니다. 전체 자료에서 모든 단어를 사용하면 모델이 한 번만 사용되는 단어의 학습 및 방향을 설계하도록 할 수 있습니다. 텍스트 코퍼스에서 많은 단어가 한두 번만 사용되는 것을 발견할 수 있습니다. 또한 텍스트에 사용한 감정은 전반적인 감정에 미미한 영향을 미치지 않으므로 일반적으로 모델에 사용하지 않아도 됩니다. num_words 매개변수를 사용하여 원하는 단어 수에 맞게 모델을 조정할 수 있습니다.

여기서 작은 숫자는 더 작고 빠른 모델을 제공할 수 있지만, 더 작은 단어를 인식하기 때문에 정확도가 떨어질 수 있습니다. 여기서 숫자가 클수록 모델이 더 크고 느려집니다. 최고의 위치를 찾는 것이 핵심입니다. 

✨ num_word 매개변수는 훈련 데이터에서 가장 자주 나타나는 단어의 개수를 지정하는데 사용한다.

### wordvec_dim 매개변수
wordved_dim 매개변수는 각 단어의 벡터에 사용할 차원의 수입니다. 연구에 따르면 결정된 단어는 수의 네 번째 루트입니다. 예를 들어 단어 2000개를 사용하는 경우 처음에는 7로 시작하는 것이 좋습니다. 사용하는 단어 수를 변경하는 경우에도 이를 변경할 수 있습니다.

### seq_len 매개변수
모델은 입력 값과 관련하여 일반적으로 매우 엄격합니다. 언어 모델의 경우 언어 모델은 특정 정적 길이에 해당하는 문장을 분류할 수 있습니다. 이는 seq_len 매개변수 또는 시퀀스 길이에 따라 결정됩니다.
단어를 숫자 (또는 토큰)로 전환하면 문장이 이러한 토큰의 시퀀스가 됩니다. 이 경우 20개의 토큰이 있는 문장을 분류하고 인식하도록 모델이 학습합니다. (이 경우) 문장이 이보다 길면 잘립니다. 더 짧으면 패딩됩니다. 코퍼스에 사용할 전용 <PAD> 토큰이 코퍼스에 표시됩니다.

## 6.데이터 로더 사용
이전에 CSV 파일을 다운로드한 경우 이제 데이터 로더를 사용하여 모델이 인식할 수 있는 학습 데이터로 변화할 차례입니다.

In [ ]:
data = DataLoader.from_csv(
    filename=data_file,
    text_column='commenttext',
    label_column='spam',
    model_spec=spec,
    delimiter=',',
    shuffle=True,
    is_training=True
)

train_data, test_data = data.split(0.9)

편집기에서 CSV 파일을 열면 각 행에 2개의 값이 있으며 파일의 첫 번째 줄에 텍스트가 기술되어 있습니다. 일반적으로 각 항목은 열로 간주됩니다.

첫 번째 열의 설명자는 commenttext이고 각 줄의 첫 번째 항목은 주석 텍스트인 것을 확인할 수 있습니다. 마찬가지로 두 번째 열의 설명자는 spam이며 각 행의 두 번째 항목은 True 또는 False,로 표시되어 텍스트가 댓글 스팸으로 간주되는지 나타냅니다. 다른 속성은 이전에 만든 model_spec를 구분 문자와 함께 설정합니다. 이 경우 파일은 쉼표로 구분되므로 쉼표입니다. 이 데이터를 모델 학습에 사용하므로 is_Training가 True로 설정됩니다.

모델 테스트를 위해 데이터의 일부를 보류하는 것이 좋습니다. 90% 는 학습용, 나머지 10% 는 테스트/평가용 이를 위해 테스트 데이터가 데이터 세트의 '하위' 10% 가 아니라 무작위로 선택되어 있는지 확인하려고 하므로 데이터를 로드할 때 무작위로 shuffle=True를 사용합니다.

## 7.모델 빌드
다음 셀은 모델을 빌드하기 위한 것으로 한 줄의 코드입니다.

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=50, validation_data=test_data)

28/28 [==============================] - 2s 32ms/step - loss: 0.6575 - accuracy: 0.7969 - val_loss: 0.6352 - val_accuracy: 0.7100
Epoch 2/2
28/28 [==============================] - 0s 10ms/step - loss: 0.6062 - accuracy: 0.8013 - val_loss: 0.5781 - val_accuracy: 0.7100
Epoch 3/3
28/28 [==============================] - 0s 8ms/step - loss: 0.5474 - accuracy: 0.8337 - val_loss: 0.5153 - val_accuracy: 0.7900
Epoch 4/4
28/28 [==============================] - 0s 10ms/step - loss: 0.4730 - accuracy: 0.8772 - val_loss: 0.4513 - val_accuracy: 0.8800
Epoch 5/5
28/28 [==============================] - 0s 10ms/step - loss: 0.4168 - accuracy: 0.8828 - val_loss: 0.3923 - val_accuracy: 0.9300
Epoch 6/6
28/28 [==============================] - 0s 7ms/step - loss: 0.3567 - accuracy: 0.8895 - val_loss: 0.3345 - val_accuracy: 0.9600
Epoch 7/7
28/28 [==============================] - 0s 7ms/step - loss: 0.3068 - accuracy: 0.9252 - val_loss: 0.2860 - val_accuracy: 0.9600
Epoch 8/8
28/28 [================

❗ 가이드라인에 따라 위 코드 실행 시,
"AttributeError: module 'tensorflow_examples.lite.model_maker.core.task.metadata_writers.text_classifier' has no attribute 'create' site:stackoverflow.com" 라는 오류가 발생했다.

2번에 "from tflite_model_maker import text_classifier" 추가하고 실행하니 정상적으로 작동한다.


머신러닝의 기본 원칙은 패턴 일치의 한 형태입니다. 처음에는 단어에 대해 선행 학습된 가중치를 로드한 후 함께 그룹화했을 때 스팸을 표시하고 그렇지 않을 단어를 예측하여 그룹화하려고 시도합니다. 처음에는 모델이 시작만 50:50에 가까워질 가능성이 높습니다.

그런 다음 테스트 결과를 측정하고 최적화 코드를 실행하여 예측을 조정한 후 다시 시도합니다. 에포크입니다. 따라서 epochs=50을 지정하면 '루프'가 50번 진행됩니다.

50세대가 되면 모델이 훨씬 더 높은 정확도로 보고됩니다. 이 경우 99%가 표시됩니다.

오른쪽에 유효성 검사 정확성 수치가 표시됩니다. 이는 일반적으로 모델이'본 적이 없는' 데이터를 분류하는 방법을 나타내기 때문에 학습 정확성보다 약간 낮습니다. 이전에 설정한 10% 테스트 데이터를 사용합니다.

## 8.모델 내보내기
학습이 완료되면 모델을 내보낼 수 있습니다.

TensorFlow는 자체 형식으로 모델을 학습시키며, 모바일 앱에서 사용하려면 TFLITE 형식으로 변환해야 합니다. Model Maker는 이러한 이러한 복잡성을 자동으로 처리합니다.

디렉터리를 지정하여 모델을 내보내면 됩니다.

In [ ]:
model.export(export_dir='mm_spam')

✨ 모델 파일은 새로 생성된 mm_spam 디렉터리에 저장된다.